In [3]:
from python_lib.modules import *
import torch
import numpy as np
import os
from python_lib.saveasfile import *

c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html


In [4]:
input_feats = torch.rand([1,2,64]).to("cuda")
input_feats_np = input_feats.cpu().detach().numpy()
input_feats_np = np.reshape(input_feats_np, (2,64))
print(input_feats_np)
dim = input_feats_np.shape

flatten_inputs = input_feats_np.flatten()
with open(os.path.join("ECAPAweights", f"ecapainput_2x64.bin"), "wb") as f:
            # Write the dimensions down
            f.write(np.array(dim, dtype=np.int32).tobytes())
            # Write the flatten bias down
            f.write(flatten_inputs.tobytes())

[[0.84810144 0.38021284 0.07742941 0.03554708 0.41127974 0.27814382
  0.97178304 0.38108122 0.6306245  0.01527929 0.7848532  0.8927886
  0.6942344  0.44062507 0.41166884 0.5559074  0.7124796  0.28590155
  0.38814515 0.62809277 0.5547842  0.9216363  0.68185574 0.57008463
  0.96388793 0.08997792 0.45683253 0.98440856 0.37720507 0.24197215
  0.14635384 0.8134315  0.6730053  0.9690083  0.1192556  0.41332304
  0.55238444 0.98969287 0.1591286  0.84837675 0.3805551  0.77151245
  0.2889573  0.647639   0.9276624  0.09597516 0.55767703 0.84730256
  0.9903024  0.14924139 0.4915164  0.31616527 0.60583097 0.5552245
  0.4263553  0.74619174 0.41627967 0.03570694 0.29239243 0.7026873
  0.64489114 0.4367838  0.790554   0.4574638 ]
 [0.6869116  0.269467   0.69392604 0.61449677 0.10298812 0.9795513
  0.9872673  0.27054292 0.0239026  0.5107536  0.8755867  0.14174664
  0.6716706  0.6615714  0.9821563  0.42259187 0.35205686 0.64875793
  0.00193101 0.3156858  0.12059551 0.8191572  0.34104615 0.5185388
  0.39

In [3]:
model = ECAPA_TDNN(input_size = 2, channels=[8,8,8,8,16], lin_neurons=6, device = "cuda").to("cuda")

BlockSave(model.return_layers(), "fullecapa", "ECAPAweights")


In [4]:
model.eval()

model(input_feats)

tensor([[0.1610, 0.1526, 0.1580, 0.1804, 0.1755, 0.1725]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [5]:
import math

def get_padding_elem(L_in: int, stride: int, kernel_size: int, dilation: int):
    """This function computes the number of elements to add for zero-padding.

    Arguments
    ---------
    L_in : int
    stride: int
    kernel_size : int
    dilation : int

    Returns
    -------
    padding : int
        The size of the padding to be added
    """
    if stride > 1:
        padding = [math.floor(kernel_size / 2), math.floor(kernel_size / 2)]

    else:
        L_out = (
            math.floor((L_in - dilation * (kernel_size - 1) - 1) / stride) + 1
        )
        padding = [
            math.floor((L_in - L_out) / 2),
            math.floor((L_in - L_out) / 2),
        ]
    return padding

get_padding_elem(64,1,5,1)

[2, 2]